# Load modules and 3D grid of peculiar velocity

In [1]:
## Load the modules

from scipy.interpolate import interpn
import healpy as hp
import numpy as np
import numexpr as ne
from astropy.coordinates import SkyCoord
import astropy.units as au

import sys
sys.path.append('../code/')
import source
import importlib
importlib.reload(source)
from source.catalog import CatalogASCII
from source.skymap import PlanckMap
from source.cosmology import FlatLambdaCDMCosmo
from source.galaxybox import Box

## Set path and load data

data_path = '../data/velocity/'
vfield_file = data_path + 'vfield_output_2000.npy'
vfield = np.load(vfield_file)

## Check data
print(vfield.shape)
print(vfield.mean())
print(vfield.std())
print(vfield.min())
print(vfield.max())

(256, 256, 256, 3)
0.0006478725489079893
246.10231630299057
-1483.860930418576
1412.1464473171718


# Load the maxBCG clusters within BOSS SDSS-3 volume

In [2]:
## Load clusters

catFile='../data/cluster/maxBCG_ID_redshift_sky_selected.dat'
cat = CatalogASCII(catFile,use_photoz=False,specz_col=3,N200_col=4)

ID = cat.id
z = cat.specz
ra = cat.coord.ra
dec = cat.coord.dec
N200 = cat.N200

# Convert cluster positions to cartesian coordinates

In [3]:
## Define cosmological model

model = FlatLambdaCDMCosmo()
model.get_astropy_cosmo()


## Convert coordinates

### Convert redshift and ra, dec to comoving Cartesian coordinates

r = model.cosmo.comoving_distance(z)
coord = SkyCoord(ra=ra, dec=dec, distance=r)
x = np.float64(coord.cartesian.x.to_value(au.Mpc))
y = np.float64(coord.cartesian.y.to_value(au.Mpc))
z = np.float64(coord.cartesian.z.to_value(au.Mpc))

### Shift the cartesian coordinates to be consistent w.r.t. BORG observer

pos_in_maxBCG_physcoord = np.stack((x, y, z), axis=-1)
pos_in_borg_physcoord = np.empty_like(pos_in_maxBCG_physcoord)
borg_box = Box()
corner = borg_box.box_corners.to_value(au.Mpc)
ne.evaluate('pos - corner', dict(pos=pos_in_maxBCG_physcoord,corner=corner[None,:]), out=pos_in_borg_physcoord)

### Copy velocity grid and object position into Box() object
borg_box.copy_velocity_grid(vfield)
borg_box.load_object_onto_grid(pos_in_borg_physcoord)

True

# Interpolate cluster 3D velocity, then project onto the LOS of each cluster

In [4]:
## Interpolate 3D velocity

borg_box.get_object_velocity()

## Project onto LOS

vlos = borg_box.project_vlos()

## Check results

print(vlos.mean())
print(vlos.std())
print(vlos.min())
print(vlos.max())

-19.173149694902538
237.64029652691644
-741.8805214107871
690.2312105322125
